In [1]:
import sys
sys.path.append("..")

In [2]:
import matplotlib.pyplot as plt

In [3]:
from squlearn.qnn.loss import *
from squlearn.qnn.training import *


In [31]:
X, y = make_blobs(60, centers=2, random_state=0)
X = MinMaxScaler((-0.9, 0.9)).fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

In [39]:
X.shape, y.shape

((60, 2), (60,))

In [41]:
 X[:,0], y

(array([-0.18035208, -0.9       ,  0.17814953,  0.43289935,  0.9       ,
         0.09406615,  0.61995605,  0.58620911, -0.29032608,  0.3747259 ]),
 array([0, 1, 0, 1, 1, 0, 1, 0, 0, 1]))

In [56]:
from squlearn import Executor
from squlearn.encoding_circuit import ChebyshevRx
from squlearn.observables import SummedPaulis
from squlearn.qnn import QNNClassifier, SquaredLoss
from squlearn.optimizers import SLSQP
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X, y = make_blobs(10, centers=2, random_state=0)
X = MinMaxScaler((-0.9, 0.9)).fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X[:,0], y, test_size=0.33, random_state=42
)

num_qubits = 1
num_features = 1
num_layers = 2 

clf = QNNClassifier(
    encoding_circuit = ChebyshevRx(num_qubits, num_features, num_layers),
    operator = SummedPaulis(num_qubits),
    executor = Executor(),
    loss = SquaredLoss(),
    optimizer = SLSQP(),
    param_ini = np.random.rand(2*num_qubits*num_layers),
    param_op_ini = np.random.rand(num_qubits+1)
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)





















fit:   6%|▌         | 6/100 [05:50<1:31:29, 58.40s/it]






















































































































































































In [57]:
circuit = ChebyshevRx(num_qubits, num_features, num_layers)
observable = SummedPaulis(num_qubits) #observable.num_parameters = num_qubits+1
param_ini = np.random.rand(2*num_qubits*num_layers)
param_op_ini = np.random.rand(num_qubits+1)
executor = Executor("statevector_simulator")

qnn_ = QNN(circuit, observable, executor, result_caching=False, optree_caching=False)

In [58]:
param_ini.dtype, param_op_ini.dtype

(dtype('float64'), dtype('float64'))

In [61]:
X_train.dtype

dtype('float64')

In [65]:
X_train = np.linspace(-0.9, 0.9, 10).reshape(-1, 1)
output_f = qnn_.evaluate("f", X_train, param_ini, param_op_ini)["f"]
output_dfdx = qnn_.evaluate("dfdx", X_train, param_ini, param_op_ini)["dfdx"]

In [66]:
def training_loss(param_ini, param_op_ini, X_train, y_train, qnn_, problem_parameters):
    g, f_initial, regularization_parameter = problem_parameters
    output_f = qnn_.evaluate("f", X_train, param_ini, param_op_ini)["f"]
    output_dfdx = qnn_.evaluate("dfdx", X_train, param_ini, param_op_ini)["dfdx"]
    return loss_function(output_f, output_dfdx, g, f_initial, regularization_parameter)

def loss_function(f_0, f_1, g, f_initial, regularization_parameter):
        """Calculates the loss function.

        f_0 = zeroth order derivative
        f_1 = first order derivative
        g = differential equation function
        Args:
            alpha_ (np.ndarray): The vector of alphas, of shape (len(x_span)+1, 

        """

        sum1 = np.sum( (f_1 - g(f_0))**2 )
        sum2 = (f_0[0] - f_initial)**2
        L = sum2 + sum1 * self.regularization_parameter
        return L

In [14]:
#QNN ODE solver 

# Calculate ground truth based on derivatives in the PINN sense (see already implemented with kernels)
# use partial_fit from qnn.qnnr.QNNRegressor, y_values will be calculated as above

# based on partial_fit, create new loss function that calculates the derivatives of the QNN

In [ ]:
def get_FQK_kernel_derivatives(self, x_array, qnn_, coef):
    """
    Get the FQK kernel and its derivatives for the given input data.

    Parameters:
    - x_array: The input data. np.array of shape (n, m) where n is the number of samples and m is the number of features.
    - qnn_: The FQK QNN.
    - coef: The coefficients of the P0 observable to be used in the QNN squlearn evaluation

    Returns:
    - output_f: The FQK kernel.
    - output_dfdx: The derivatives of the FQK kernel:  shape (n, n, m*2),  the last dimension is the derivative with respect to the input data. 
    """
    x_array = x_array.reshape(-1, 1) #reshape to column vector
    x_list_circuit_format = self.x_to_circuit_format(x_array)


    output_f = qnn_.evaluate("f", x_list_circuit_format, [], coef)["f"]
    output_dfdx = qnn_.evaluate("dfdx", x_list_circuit_format, [], coef)["dfdx"]
    

    #reshape the output to the shape of the gram matrix
    output_f = output_f.reshape((len(x_array), len(x_array)))
    output_dfdx = output_dfdx.reshape((len(x_array), len(x_array), len(x_array[0])*2))


    return output_f, output_dfdx

New squlearn

In [58]:
from squlearn.qnn.lowlevel_qnn_base import LowLevelQNNBase
from squlearn.qnn.lowlevel_qnn_qiskit import LowLevelQNNQiskit

In [ ]:
        #qnn_ = BaseQNN(FQK_Circuit, P0_, self.executor, result_caching=False, optree_caching=False)


In [69]:
encoding_circuit = ChebyshevRx(num_qubits, num_features, num_layers)
operator = SummedPaulis(num_qubits),
executor = Executor("statevector_simulator")
loss = SquaredLoss(),
optimizer = SLSQP(),
param_ini = np.random.rand(2*num_qubits*num_layers),
param_op_ini = np.random.rand(num_qubits+1)

In [78]:
LowLevelQNNQiskit?

Init signature:
LowLevelQNNQiskit(
    parameterized_quantum_circuit: squlearn.encoding_circuit.encoding_circuit_base.EncodingCircuitBase,
    operator: Union[squlearn.observables.observable_base.ObservableBase, list],
    executor: squlearn.util.executor.Executor,
    optree_caching=True,
    result_caching=True,
) -> None
Docstring:     
Low level implementation of QNNs and its derivatives based on Qiskit.

Args:
    pqc (EncodingCircuitBase) : parameterized quantum circuit in encoding circuit format
    operator (Union[ObservableBase,list]): Operator that is used in the expectation
        value of the QNN. Can be a list for multiple outputs.
    executor (Executor) : Executor that is used for the evaluation of the QNN
    optree_caching : Caching of the optree expressions (default = True recommended)
    result_caching : Caching of the result for each `x`, `param`, `param_op` combination
        (default = True)

Attributes:
-----------

Attributes:
    num_qubits (int): Number of 

AttributeError: 'tuple' object has no attribute 'set_map'

In [46]:
BaseQNN?

Init signature:
BaseQNN(
    encoding_circuit: 'EncodingCircuitBase',
    operator: 'Union[ObservableBase, list[ObservableBase]]',
    executor: 'Executor',
    loss: 'LossBase',
    optimizer: 'OptimizerBase',
    param_ini: 'Union[np.ndarray, None]' = None,
    param_op_ini: 'Union[np.ndarray, None]' = None,
    batch_size: 'int' = None,
    epochs: 'int' = None,
    shuffle: 'bool' = None,
    opt_param_op: 'bool' = True,
    variance: 'Union[float, Callable]' = None,
    shot_control: 'ShotControlBase' = None,
    parameter_seed: 'Union[int, None]' = 0,
    caching: 'bool' = True,
    pretrained: 'bool' = False,
    callback: 'Union[Callable, str, None]' = None,
    **kwargs,
) -> 'None'
Docstring:     
Base Class for Quantum Neural Networks.

Args:
    encoding_circuit : Parameterized quantum circuit in encoding circuit format
    operator : Operator that are used in the expectation value of the QNN. Can be a list for
        multiple outputs.
    executor : Executor instance
    

In [ ]:
def get_FQK_kernel_derivatives(self, x_array, qnn_, coef):
        """
        Get the FQK kernel and its derivatives for the given input data.

        Parameters:
        - x_array: The input data. np.array of shape (n, m) where n is the number of samples and m is the number of features.
        - qnn_: The FQK QNN.
        - coef: The coefficients of the P0 observable to be used in the QNN squlearn evaluation

        Returns:
        - output_f: The FQK kernel.
        - output_dfdx: The derivatives of the FQK kernel:  shape (n, n, m*2),  the last dimension is the derivative with respect to the input data. 
        """
        

        x_array = x_array.reshape(-1, 1) #reshape to column vector
        x_list_circuit_format = self.x_to_circuit_format(x_array)


        output_f = qnn_.evaluate("f", x_list_circuit_format, [], coef)["f"]
        output_dfdx = qnn_.evaluate("dfdx", x_list_circuit_format, [], coef)["dfdx"]
        

        #reshape the output to the shape of the gram matrix
        output_f = output_f.reshape((len(x_array), len(x_array)))
        output_dfdx = output_dfdx.reshape((len(x_array), len(x_array), len(x_array[0])*2))


        return output_f, output_dfdx

In [40]:
#QNN ODE solver 

# Calculate ground truth based on derivatives in the PINN sense (see already implemented with kernels)
# use partial_fit from qnn.qnnr.QNNRegressor, y_values will be calculated as above

# based on partial_fit, create new loss function that calculates the derivatives of the QNN